# nlp-homework
- 자연어처리 과제 ai융합학부 20211336 박예인



## Overview
- **Food Order Analysis Dataset**: 3,000 records
   - **Goal**: Extract food names, option names, and quantities from order sentences
- **Solution**: Fine-tuning meta-llama/Llama-3.2-1B-Instruct
- **Methodology**:
   - Load model with 4-bit quantization
   - Attach LoRA adapter
   - Train using **SFTTrainer**: Sentence -> next token prediction
   - Process dataset with **ConstantLengthDataset**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install transformers accelerate datasets peft trl bitsandbytes wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import os
from dataclasses import dataclass, field
from typing import Optional
import re

import torch
import sys
# import tyro
from accelerate import Accelerator
from datasets import load_dataset, Dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import (
    HfArgumentParser,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    TextStreamer,
    logging as hf_logging,
)
import logging
from trl import SFTTrainer, SFTConfig

from trl.trainer import ConstantLengthDataset

# Setting

## Q1. meta-llama/Llama-3.2-1B-Instruct 모델 사용
- base_model_id에 meta-llama/Llama-3.2-1B-Instruct 저장

In [4]:
base_model_id = "meta-llama/Llama-3.2-1B-Instruct"
device_map="cuda"
torch_dtype = torch.bfloat16
output_dir = "./nlp-homework"
dataset_name = "drive/MyDrive/llm-modeling-lab.jsonl"
seq_length = 512

# Raw Dataset

## Q2. train(2800개), validation(200개) dataset 분리

In [5]:
from datasets import Dataset

# JSON 데이터 로드
full_dataset = Dataset.from_json(path_or_paths=dataset_name)

# 데이터셋 분리
train_dataset = full_dataset.select(range(2800))  # 학습 데이터 2,800개
validation_dataset = full_dataset.select(range(2800, 3000))  # 검증 데이터 200개

# 확인
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(validation_dataset)}")


Generating train split: 0 examples [00:00, ? examples/s]

Train dataset size: 2800
Validation dataset size: 200


# Loading the Tokenizer

In [6]:
! pip install huggingface_hub

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id
)
tokenizer.padding_side = "right"

# Loading the Base Model

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

### base_model_id=meta-llama/Llama-3.2-1B-Instruct 모델 불러오기

In [11]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",  # {"": Accelerator().local_process_index},
)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [12]:
base_model.config.use_cache = False

In [13]:
if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training
if base_model.config.pad_token_id != tokenizer.pad_token_id:
    base_model.config.pad_token_id = tokenizer.pad_token_id

# Utilities

In [14]:
def chars_token_ratio(dataset, tokenizer, prepare_sample_text, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens

In [15]:
def function_prepare_sample_text(tokenizer, for_train=True):
    """A Closure"""

    def _prepare_sample_text(example):
        """Prepare the text from a sample of the dataset."""
        user_prompt="너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: "
        messages = [
            # {"role": "system", "content": f"{system_prompt}"},
            {"role": "user", "content": f"{user_prompt}{example['input']}"},
        ]
        if for_train:
            messages.append({"role": "assistant", "content": f"{example['output']}"})

        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False if for_train else True)
        return text
    return _prepare_sample_text

In [16]:
def create_datasets(tokenizer, dataset, seq_length):

    prepare_sample_text = function_prepare_sample_text(tokenizer)

    chars_per_token = chars_token_ratio(dataset, tokenizer, prepare_sample_text)
    print(
        f"The character to token ratio of the dataset is: {chars_per_token:.2f}"
    )

    cl_dataset = ConstantLengthDataset(
        tokenizer,
        dataset,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=seq_length,
        chars_per_token=chars_per_token,
    )

    return cl_dataset

# Build a dataset

### - 데이터셋 생성

In [17]:
# 훈련 데이터셋 생성
train_ds = create_datasets(tokenizer, train_dataset, seq_length)

# 검증 데이터셋 생성
val_ds = create_datasets(tokenizer, validation_dataset, seq_length)

100%|██████████| 400/400 [00:00<00:00, 1496.86it/s]


The character to token ratio of the dataset is: 2.61


 50%|█████     | 200/400 [00:00<00:00, 1518.73it/s]

The character to token ratio of the dataset is: 2.62


In [18]:
# 훈련 데이터셋 샘플 확인
train_it = iter(train_ds)
print(tokenizer.decode(next(train_it)['input_ids']))

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:648: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


레귤러
- 분석 결과 1: 음식명:탕수육, 옵션:소, 수량:두 그릇<|eot_id|><|eot_id|><|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 08 Dec 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.
### 주문 문장: 동참하려고 하는데요, 세트메뉴 F 하나랑 미린다 중 한 캔 주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

- 분석 결과 0: 음식명:세트메뉴 F,수량:하나
- 분석 결과 1: 음식명:미린다,옵션:중,수량:한 캔<|eot_id|><|eot_id|><|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 08 Dec 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.
### 주문 문장: 아보카도새우포케 한 그릇하고 팥빙수 한 개 주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

- 분석 결과 0: 음식명:아보카도새우포케, 수량:한 그릇
- 분석 결과 1: 음식명:팥빙수, 수량:한 개<|eot_id|><|eot_id|><|begin_of_text|><|begin_of_text|><|start_header_

In [19]:
# 검증 데이터셋 샘플 확인
val_it = iter(val_ds)
print(tokenizer.decode(next(val_it)['input_ids']))

�션명, 수량을 차례대로 추출해야 한다.
### 주문 문장: 교촌리얼후라이드 한 마리 주문할게요. 그리고 머릿고기 대를 추가할게요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

- 분석 결과 0: 음식명:교촌리얼후라이드,수량:한 마리
- 분석 결과 1: 음식명:머릿고기,옵션:대<|eot_id|><|eot_id|><|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 08 Dec 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.
### 주문 문장: 한우고기말이 두 판하고 마늘간장 소 짜로 주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

- 분석 결과 0: 음식명:한우고기말이, 수량:두 판
- 분석 결과 1: 음식명:마늘간장, 옵션:소짜<|eot_id|><|eot_id|><|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 08 Dec 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.
### 주문 문장: 참이슬 한 병으로 시작하고, 볶음면 한 그릇 추가 주세요.<|eot_id|><|start_header_id|>assistant<|end_

# Fine Tuning



## Training Time (1 Epoch)
- **T4**: 40 min

## Loss
- **500 steps**: 0.581900
- **1000 steps**: 0.500900
- **1400 steps**: 0.462000

## Seting for a LoRA adapter

In [20]:
lora_config = LoraConfig(
            r=8,
            lora_alpha=16,
            lora_dropout=0.05,
            target_modules=[
                "q_proj",
                "k_proj",
                "v_proj",
                "o_proj",
                "down_proj",
                "up_proj",
                "gate_proj",
            ],
            bias="none",
            task_type="CAUSAL_LM",
        )

In [21]:
peft_config = lora_config

In [22]:
from google.colab import userdata
import wandb

wandb_api_key = userdata.get('WANDB_API_KEY')
if wandb_api_key:
    wandb.login(key=wandb_api_key)
    print("Successfully logged in to Weights & Biases")
else:
    print("WANDB_API_KEY not found in Colab secrets")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Successfully logged in to Weights & Biases


In [23]:
wandb.init(project="nlp-llama-homework")

wandb: Currently logged in as: yenqkr (yenqkr-study). Use `wandb login --relogin` to force relogin


In [27]:
sft_config = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    max_grad_norm=0.3,
    weight_decay=0.05,
    num_train_epochs=1,
    logging_steps=20,
    eval_strategy="no",
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    max_seq_length=seq_length,
    report_to="wandb",
    # report_to="none",
    run_name="nlp-homework",

)

In [28]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=sft_config
)

In [29]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:648: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


Step,Training Loss
20,2.704600
40,2.308000
60,1.827000
80,1.284900
100,1.080100
120,0.935900
140,0.876500
160,0.847500
180,0.773300
200,0.712900


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:648: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:648: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


TrainOutput(global_step=1400, training_loss=0.649314375604902, metrics={'train_runtime': 2388.7175, 'train_samples_per_second': 1.172, 'train_steps_per_second': 0.586, 'total_flos': 8419093040332800.0, 'train_loss': 0.649314375604902, 'epoch': 1.0})

## Q3. hugging face에 파인튜닝한 LoRa 어댑터 업로드
- push_to_hub로 업로드 되도록 설정
- 모델 ID : yenqkr/nlp-homework

In [30]:
# 모델을 Hugging Face에 업로드
trainer.push_to_hub("nlp-homework")

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yenqkr/nlp-homework/commit/265b4ff20f1ec17305a5bf61ace768055d6c63f2', commit_message='nlp-homework', commit_description='', oid='265b4ff20f1ec17305a5bf61ace768055d6c63f2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yenqkr/nlp-homework', endpoint='https://huggingface.co', repo_type='model', repo_id='yenqkr/nlp-homework'), pr_revision=None, pr_num=None)

# Validation

## Q4. 허깅페이스에 올린 어댑터 모델 불러오기
- model_id = "yenqkr/nlp-homework"

In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Hugging Face Hub에 업로드한 모델 ID
model_id = "yenqkr/nlp-homework"  # 업로드한 모델의 ID

model = (
    AutoPeftModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        # device='cuda',
        trust_remote_code=True,
    )
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

adapter_config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

# Evaluation

## Utilities for Evaluation

In [32]:
def wrapper_generate(tokenizer, model, input_prompt, do_stream=False):
    def get_text_after_prompt(text):
        pattern = r'<start_of_turn>model\n(.*?)<end_of_turn>'
        match = re.search(pattern, text, re.DOTALL)

        if match:
            extracted_text = match.group(1).strip()
            return extracted_text
        else:
            return text


    data = tokenizer(input_prompt, return_tensors="pt")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    input_ids = data.input_ids[..., :-1]
    with torch.no_grad():
        pred = model.generate(
            input_ids=input_ids.cuda(),
            streamer=streamer if do_stream else None,
            use_cache=True,
            max_new_tokens=128, #float("inf"),
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    decoded_text = tokenizer.batch_decode(pred, skip_special_tokens=False)
    generated_text = decoded_text[0]

    # "<|eot_id|>"와 같은 불필요한 태그 제거
    cleaned_text = re.sub(r'<\|.*?\|>', '', generated_text).strip()

    # 분석 결과만 추출
    return extract_analysis_results(cleaned_text)

In [33]:
def extract_analysis_results(text):
    # 정규식을 사용하여 "분석 결과" 부분만 추출
    pattern = r'- 분석 결과 \d+: .*'
    matches = re.findall(pattern, text)
    if matches:
        return '\n'.join(matches)  # 매칭된 결과를 줄바꿈으로 연결
    else:
        return "분석 결과가 없습니다."


## Release GPU memory

In [ ]:
import gc
del trainer
del base_model
gc.collect()
torch.cuda.empty_cache()

If you want other checkpoints, modify suffix.

## Test

In [34]:
preprocessor = function_prepare_sample_text(tokenizer, for_train=False)

In [35]:
# 검증 데이터셋의 input값 불러오기
inputs = [item["input"] for item in validation_dataset]
inputs

['꽃살버섯샐러드 한그릇과 스테이크올리오스파게티 세트를 주세요.',
 '교촌통통치킨카츠 한 판, 행복지수커피 한잔, 그리고 헤즐넛 핫 하나도 주세요.',
 '에스프레소도피오(핫) 한잔과 진하게 허니티 한 잔 주세요.',
 '밀크티 두 캔에, 도토리묵밥 한 그릇이랑 동치미 한병 주세요.',
 '열무국수 한 그릇 곱배기로 주시고, 칼국수도 한 뚝배기요.',
 '식당에서 제일 잘나가는 "민족치킨" 하나 주세요, 그리고 토란줄기 감자탕도 하나 추가요.',
 '오늘 저녁은 참치모듬 하나랑 제육두루치기 둘 주세요.',
 '구슬바닐라카페스무디 ICE 한잔이랑 준과랑치즈케익 하나 주세요.',
 '라디오더치라떼 ICE 한잔하고요, 볼케이노피자 R 사이즈 한판 주세요.',
 '물회 특 한그릇 대로 해주시고, 웨이퍼롤 바닐라 세트로 주세요.',
 '다크쇼콜라 L 사이즈 한잔 해주시고, 짜파게티 한개 주세요.',
 '코다리찜 중 사이즈 주시고, 클라우드라떼 S 사이즈 한잔 주세요.',
 '제주두루치기 2인분 주시고, B set 800g으로 주세요.',
 '오울블루 한그릇 주시고, 라면에 계란 추가해서 주세요.',
 '돈가스덮밥 등심 한그릇 주시고, 심심풀이새우깡 하나 주세요.',
 '금준미홍차 한잔하고, 까망베르치즈구이 하나 주시고, 젤라또 초코 한개 주세요.',
 '불목살 중자 200g와 꼬치어묵 세 개, 흑당버블밀크티 아이스 레귤러 사이즈 두 잔 주세요.',
 '오늘은 호두아몬드파티 24cm 한 판, 블루베리스무디 XL사이즈 두 잔으로 할게요.',
 '모두가 좋아하는 돈돈탕 중자 한 그릇, 옥수수 한 개 주세요.',
 '카라멜프라페 그란데 사이즈 한 잔, 닭불고기백반 1개 주세요.',
 '징기스칸 양고기 220g하고 사리 당면 한 개 주시고, 아이스 말차라떼 이잔 주세요.',
 '보쌈 세트 한 판, 리얼 고구마라떼 라지 사이즈로 한 잔 주세요.',
 '치즈크레커 두 판, 레몬에이드 ICE 레귤러 한 잔 주세요.',
 '오도독살 180g, 빵 추가 한개 주세요.',
 '한방 대나

In [36]:
# 검증 데이터셋의 output값 불러오기
expected_outputs = [item["output"] for item in validation_dataset]
expected_outputs

['- 분석 결과 0: 음식명:꽃살버섯샐러드, 수량:한그릇 \n- 분석 결과 1: 음식명:스테이크올리오스파게티',
 '- 분석 결과 0: 음식명:교촌통통치킨카츠, 수량: 한 판\n- 분석 결과 1: 음식명:행복지수커피, 수량: 한잔\n- 분석 결과 2: 음식명:헤즐넛, 옵션: 핫, 수량: 하나',
 '- 분석 결과 0: 음식명:에스프레소도피오,옵션:핫,수량:한잔\n- 분석 결과 1: 음식명:허니티,옵션:진하게,수량:한 잔',
 '- 분석 결과 0: 음식명:밀크티,수량:두 캔\n- 분석 결과 1: 음식명:도토리묵밥, 수량: 한 그릇\n- 분석 결과 2: 음식명:동치미, 수량: 한병',
 '- 분석 결과 0: 음식명:열무국수,옵션:곱배기, 수량:한 그릇\n- 분석 결과 1: 음식명:칼국수, 수량:한 뚝배기',
 '- 분석 결과 0: 음식명:민족치킨, 수량:하나\n- 분석 결과 1: 음식명:토란줄기 감자탕, 수량:하나',
 '- 분석 결과 0: 음식명:참치모듬,수량:하나\n- 분석 결과 1: 음식명:제육두루치기,수량:둘',
 '- 분석 결과 0: 음식명:구슬바닐라카페스무디,옵션:ICE,수량:한잔\n- 분석 결과 1: 음식명:준과랑치즈케익,수량:하나',
 '- 분석 결과 0: 음식명:라디오더치라떼,옵션:ICE,수량:한잔\n- 분석 결과 1: 음식명:볼케이노피자,옵션:R,수량:한판',
 '- 분석 결과 0: 음식명:물회,옵션:특,수량:한그릇\n- 분석 결과 1: 음식명:웨이퍼롤,옵션:바닐라',
 '- 분석 결과 0: 음식명:다크쇼콜라,옵션:L,수량:한잔\n- 분석 결과 1: 음식명:짜파게티,수량:한개',
 '- 분석 결과 0: 음식명:코다리찜,옵션:중\n- 분석 결과 1: 음식명:클라우드라떼,옵션:S,수량:한잔',
 '- 분석 결과 0: 음식명:제주두루치기,수량:2인분\n- 분석 결과 1: 음식명:B set,옵션:800g',
 '- 분석 결과 0: 음식명:오울블루,수량:한그릇\n- 분석 결과 1: 음식명:라면,옵션:계란추가',
 '- 분석 결과 0: 음식명:돈가스덮밥,

### 검증 데이터셋의 input 값을 훈련시킨 모델에 입력시켜 output 생성

In [37]:
# 검증 데이터셋의 input 값을 훈련시킨 모델에 입력시켜 결과 추출하기
generated_outputs = []
for input_text in tqdm(inputs, desc="Generating Outputs"):
    print(f"Input Text: {input_text}")  # 입력 확인
    result = wrapper_generate(
        tokenizer=tokenizer,
        model=model,
        input_prompt=preprocessor({'input': input_text}),
        do_stream=False  # Stream 끄기
    )
    print(f"Result: {result}")  # 결과 확인
    generated_outputs.append(result)


Generating Outputs:   0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Input Text: 꽃살버섯샐러드 한그릇과 스테이크올리오스파게티 세트를 주세요.


Generating Outputs:   0%|          | 1/200 [00:08<27:23,  8.26s/it]

Result: - 분석 결과 0: 음식명:꽃살버섯샐러드,수량:한그릇
- 분석 결과 1: 음식명:스테이크올리오스파게티,수량:세트
- 분석 결과 2: 음식명:꽃살버섯샐러드,옵션:스파게티,수량:한그릇
- 분석 결과 3: 음식명:스테이크올리오스파게티,수량:세트
Input Text: 교촌통통치킨카츠 한 판, 행복지수커피 한잔, 그리고 헤즐넛 핫 하나도 주세요.


Generating Outputs:   1%|          | 2/200 [00:13<21:32,  6.53s/it]

...

Result: - 분석 결과 0: 음식명:콩국수,옵션:미디엄,수량:한 그릇
- 분석 결과 1: 음식명:갈비레오,옵션:라지,수량:두 판.
Input Text: 왕갈비탕 특 한 판, 마치찜 대 한 그릇 주세요.


Generating Outputs: 100%|█████████▉| 199/200 [14:24<00:03,  3.84s/it]

Result: - 분석 결과 0: 음식명:왕갈비탕,옵션:특,수량:한 판
- 분석 결과 1: 음식명:마치찜,옵션:대,수량:한 그릇.
Input Text: 오늘은 뉴트리바 한 캔 줘. 그리고 불백 곱배기 한 그릇 같이 주세요.


Generating Outputs: 100%|██████████| 200/200 [14:28<00:00,  4.34s/it]

Result: - 분석 결과 0: 음식명:뉴트리바, 수량:한 캔
- 분석 결과 1: 음식명:불백, 옵션:곱배기, 수량:한 그릇.


### 생성된 output 값들

In [38]:
generated_outputs

['- 분석 결과 0: 음식명:꽃살버섯샐러드,수량:한그릇\n- 분석 결과 1: 음식명:스테이크올리오스파게티,수량:세트\n- 분석 결과 2: 음식명:꽃살버섯샐러드,옵션:스파게티,수량:한그릇\n- 분석 결과 3: 음식명:스테이크올리오스파게티,수량:세트',
 '- 분석 결과 0: 음식명:교촌통통치킨카츠, 수량:한 판\n- 분석 결과 1: 음식명:행복지수커피, 수량:한잔\n- 분석 결과 2: 음식명:헤즐넛, 옵션:핫, 수량:하나.',
 '- 분석 결과 0: 음식명:에스프레소도피오,옵션:핫,수량:한잔\n- 분석 결과 1: 음식명:허니티,옵션:진하게,수량:한 잔.',
 '- 분석 결과 0: 음식명:밀크티,수량:두 캔\n- 분석 결과 1: 음식명:도토리묵밥,수량:한 그릇\n- 분석 결과 2: 음식명:동치미,수량:한병.',
 '- 분석 결과 0: 음식명:열무국수,옵션:곱배기,수량:한 그릇\n- 분석 결과 1: 음식명:칼국수,옵션:뚝배기,수량:한 뚝배기.',
 '- 분석 결과 0: 음식명:민족치킨, 수량:하나\n- 분석 결과 1: 음식명:토란줄기 감자탕, 수량:하나.',
 '- 분석 결과 0: 음식명:참치모듬, 수량:하나\n- 분석 결과 1: 음식명:제육두루치기, 수량:둘',
 '- 분석 결과 0: 음식명:구슬바닐라카페스무디,옵션:ICE,수량:한잔\n- 분석 결과 1: 음식명:준과치즈케익,수량:하나.',
 '- 분석 결과 0: 음식명:라디오더치라떼,옵션:ICE,수량:한잔\n- 분석 결과 1: 음식명:볼케이노피자,옵션:R,수량:한판.',
 '- 분석 결과 0: 음식명:물회,옵션:특,수량:한그릇\n- 분석 결과 1: 음식명:웨이퍼롤 바닐라 세트,수량:세트',
 '- 분석 결과 0: 음식명:다크쇼콜라,옵션:L,수량:한잔\n- 분석 결과 1: 음식명:짜파게티,수량:한개',
 '- 분석 결과 0: 음식명:코다리찜,옵션:중\n- 분석 결과 1: 음식명:클라우드라떼,옵션:S,수량:한잔.',
 '- 분석 결과 0: 음식명:제주두루치기, 수량:2인분\n- 분석 결과 1: 음식명:B set, 

## Q5. sacrebleu 를 사용하여 검증 세트와 생성된 출력 간의 BLEU 점수를 측정

In [39]:
! pip install evaluate
! pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.3 MB/s eta 0:00:00


In [40]:
import evaluate
bleu_metric = evaluate.load("sacrebleu")

In [41]:
# 검증 데이터셋의 output과 생성된 output간의 BLEU score 확인
bleu_results = bleu_metric.compute(predictions=generated_outputs, references=expected_outputs)
bleu_results

{'score': 79.19589065406393,
 'counts': [5414, 5085, 4765, 4451],
 'totals': [6511, 6311, 6111, 5911],
 'precisions': [83.15158961757027,
  80.57360164791633,
  77.97414498445426,
  75.30028759939097],
 'bp': 1.0,
 'sys_len': 6511,
 'ref_len': 5567}